# Import

In [0]:
# Import poalrs
import polars as pl
# Import databricks essentials
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window as W

from datetime import datetime, timedelta

import uuid
import shutil
import os
import glob
import functools
import polars as pl
from pyspark.sql import DataFrame as SparkDataFrame, SparkSession

# Define project volume
CATALOG_NAME = "main"
SCHEMA_NAME = "default"
VOLUME_NAME = "my_test_volume"

spark = SparkSession.builder.appName("scaffold").getOrCreate()

# Define class

In [0]:
class VolumeSpiller:
    def __init__(self, spark: SparkSession, catalog: str, schema: str, volume_name: str, is_dev: bool = False):
        self.spark = spark
        self.is_dev = is_dev
        self.full_name = f"{catalog}.{schema}.{volume_name}"
        self.volume_root = f"/Volumes/{catalog}/{schema}/{volume_name}"
        
        if is_dev:
            # In Dev, we just want to make sure it exists so we don't lose data
            self.spark.sql(f"CREATE VOLUME IF NOT EXISTS {self.full_name}")
        else:
            # In Prod, we want a clean slate, so we drop then create
            self.spark.sql(f"DROP VOLUME IF EXISTS {self.full_name}")
            self.spark.sql(f"CREATE VOLUME {self.full_name}")
            
        self._active_temp_dirs = []

    def get_path(self, name: str) -> str:
        """Returns the absolute path for a named folder in the volume."""
        return f"{self.volume_root}/{name.lstrip('/')}"

    def save_checkpoint(self, df: pl.DataFrame | pl.LazyFrame, name: str):
        """
        Saves a Polars DataFrame/LazyFrame as a named checkpoint (directory).
        Automates folder creation and file naming.
        """
        checkpoint_dir = self.get_path(name)
        os.makedirs(checkpoint_dir, exist_ok=True)
        
        # Standardizing on data.parquet inside the folder
        target_path = f"{checkpoint_dir}/data.parquet"
        
        if isinstance(df, pl.LazyFrame):
            df.sink_parquet(target_path, compression="zstd")
        else:
            df.write_parquet(target_path, compression="zstd")
        
        print(f"✅ Checkpoint '{name}' saved to UC Volume.")

    def load_checkpoint(self, name: str, eager: bool = True) -> pl.DataFrame | pl.LazyFrame:
        """
        Loads a named checkpoint into Polars.
        Uses glob patterns to stay immune to Spark/Polars metadata differences.
        """
        checkpoint_dir = self.get_path(name)
        glob_path = f"{checkpoint_dir}/*.parquet"
        
        if not os.path.exists(checkpoint_dir):
            raise FileNotFoundError(f"Checkpoint '{name}' not found at {checkpoint_dir}")

        return pl.read_parquet(glob_path) if eager else pl.scan_parquet(glob_path)

    def teardown(self):
        """Cleans up. In Dev, we keep the data for inspection."""
        if self.is_dev:
            print(f"🛠️ DEV MODE: Data preserved at {self.volume_root}")
        else:
            self.spark.sql(f"DROP VOLUME IF EXISTS {self.full_name}")
            print("🗑️ PROD MODE: Volume dropped.")

    def spark_to_polars(self, df: SparkDataFrame, cleanup: bool = True, eager: bool = True, optimize_files: bool = True) -> pl.DataFrame | pl.LazyFrame:
        run_id = uuid.uuid4().hex
        temp_dir = self.get_path(f"spill_{run_id}")
        
        if not (cleanup and eager):
            self._active_temp_dirs.append(temp_dir)

        try:
            if optimize_files:
                df = df.coalesce(4) 

            # Spark writes the directory
            df.write.mode("overwrite").option("compression", "zstd").parquet(temp_dir)

            # FIX: Use a glob pattern to only read the .parquet files, ignoring Spark metadata
            glob_path = f"{temp_dir}/*.parquet"
            
            if eager:
                return pl.read_parquet(glob_path)
            else:
                return pl.scan_parquet(glob_path)

        finally:
            if cleanup and eager:
                shutil.rmtree(temp_dir, ignore_errors=True)

    def polars_to_spark(self, df: pl.DataFrame | pl.LazyFrame, cleanup: bool = True) -> SparkDataFrame:
        run_id = uuid.uuid4().hex
        temp_dir = self.get_path(f"spill_rev_{run_id}")
        
        if not cleanup: 
             self._active_temp_dirs.append(temp_dir)

        try:
            # Ensure the directory exists
            os.makedirs(temp_dir, exist_ok=True)
            # We name the file 'part-0.parquet' inside the folder to mimic Spark behavior
            file_path = f"{temp_dir}/part-0.parquet"
            
            if isinstance(df, pl.LazyFrame):
                df.sink_parquet(file_path, compression="zstd")
            else:
                df.write_parquet(file_path, compression="zstd")

            # Spark reads the directory, not the file
            spark_df = self.spark.read.parquet(temp_dir)
            
            if cleanup:
                spark_df.cache().count() 
                
            return spark_df

        finally:
            if cleanup:
                shutil.rmtree(temp_dir, ignore_errors=True)

# Use class

In [0]:
# --- CONFIGURATION ---
IS_DEV = False  # Toggle this to False for production runs
CATALOG = "main"
SCHEMA = "default"
VOL_NAME = "my_pipeline_dev" if IS_DEV else f"vol_{uuid.uuid4().hex[:8]}"

spiller = VolumeSpiller(spark, CATALOG, SCHEMA, VOL_NAME, is_dev=IS_DEV)

try:
    SCHEMA_NAME = T.StructType([
        T.StructField("transaction_id", T.IntegerType(), False),
        T.StructField("customer_name", T.StringType(), True),
        T.StructField("product_category", T.StringType(), True),
        T.StructField("amount", T.DoubleType(), True),
        T.StructField("timestamp", T.TimestampType(), True)
    ])

    data = []
    categories = ["Electronics", "Home", "Garden", "Books", "Toys"]
    names = ["Alice", "Bob", "Charlie", "Diana", "Edward"]

    for i in range(1, 21):
        data.append((
            i, 
            names[i % 5], 
            categories[i % 5], 
            round(10.5 * i, 2), 
            datetime(2024, 1, 1) + timedelta(days=i)
        ))

    my_spark_df = spark.createDataFrame(data, SCHEMA_NAME)
    
    # If we are in dev, maybe we don't clean up so we can check the file
    pl_df = spiller.spark_to_polars(my_spark_df, cleanup=(not IS_DEV))
    
    spiller.save_checkpoint(pl_df, "cleaned_data")

finally:
    # This will only actually delete if IS_DEV is False
    spiller.teardown()